In [ ]:
!pip install --upgrade 'qiskit[visualization]'

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import numpy as np
from docplex.mp.model import Model
from math import log

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

# ETF Optimization (Use_case_v2) 


In [2]:
# Quantum Aggregator Wrapper

# Import algorithms 
import grovers_search as grovers
import optim_wrapper as optimization


def aggregator(algorithm, dict_details):
    if algorithm == 'grovers':
        result = grovers.grovers_search(dict_details)
    if algorithm == 'optimizer':
        result = optimization.optimize_portfolio(dict_details)
    return result


## Binary representation of integers limited to budget amount

(Based on H2 Hypothesis in paper v1 and stock grouping in paper v2)

$\begin{aligned}
\min_{x \in \mathbb{N}^n}  q x^T \Sigma x - \mu^T x\\
\end{aligned}$

subject to:

$\begin{aligned}
\ \sum_{i=0}^{n} S^T x \le B,\quad x \in \mathbb{N}^n,\quad S \in \mathbb{R}^n,\quad B \in \mathbb{R} \qquad \approx_{\big|Variance(S) \to 0}  \qquad \sum_{i=0}^{n} x \le \left\lfloor \frac{1}{\overline{S}}B \right\rfloor,\quad x \in \mathbb{N}^n,\quad \lfloor.\rfloor \in \mathbb{N}\\
\end{aligned}$

integer to binary variable change:

$\begin{aligned}
\ x = \sum_{i=0}^{log_{2}(\frac{1}{\overline{S}}B)}2^iy_i,\quad y \in \{0,1\}
\end{aligned}$

where we use the following notation:

- $x \in \mathbb{N}^n$ denotes the vector of integer decision variables, which indicate which assets to pick and how much ($x[i] > 0$) and which not to pick ($x[i] = 0$),
- $y$ represents the binary decomposition of $x$
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- $S$ defined as the prices related to the stock ( $\overline{S}$ is the mean of the prices),
- $B$ denotes the budget, i.e. the maximum amount of money to be spent purchasing assets.
- $\lfloor.\rfloor$ is the integer part operator


In [3]:
# prepare problem instance
n = 5            # number of assets
q = 0.5          # risk factor
budget = 2250  # budget
# automatic penalty
#penalty = budget**n    # scaling of penalty term
#print('Penalty:', penalty)

In [4]:

# example instance
s_real = np.array([60.04, 55.87, 139.72, 32.46, 111.07])
grouping = np.array([2, 3, 1, 4, 1])
s=grouping*s_real

print(' ̃s:', s)

 ̃s: [120.08 167.61 139.72 129.84 111.07]


In [5]:
print(s.mean())

133.664


In [6]:
budget_bits = int(log(budget/np.min(s),2))
print("Budget variables per asset:", budget_bits)

Budget variables per asset: 4


In [7]:
mu = np.array([0.0029, 0.00206, 0.00033, 0.00263, -0.00012])
sigma = np.array([
    [ 0.00016, 0.00014, 0.00013, 0.00016, 0.00015],
    [ 0.00014, 0.00016, 0.00013, 0.00016, 0.00014],
    [ 0.00013, 0.00013, 0.00028, 0.00013, 0.00028],
    [ 0.00016, 0.00016, 0.00013, 0.00021, 0.00015],
    [ 0.00015, 0.00014, 0.00028, 0.00015, 0.00030]
])

In [8]:
# create docplex model
mdl = Model('portfolio_optimization')



y_var = mdl.binary_var_list('y{}_{}'.format(i, j) for i in range(n) for j in range(budget_bits))

y = np.zeros([n, budget_bits])
y = y.astype('object')

var_location = np.zeros([n, budget_bits])
var_location = var_location == 0

y[var_location] = y_var

#x = (mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n))

objective = mdl.sum([mu[i]*mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) * mdl.sum([(2**k)*y[j][k] for k in range(budget_bits)]) for i in range(n) for j in range(n)])
mdl.maximize(objective)

# as we have equalizer prices, we can aprox the price as 1, and the budget as divided by mean 
bb = int(budget/s.mean())
mdl.add_constraint(mdl.sum(mdl.sum([(2**k)*y[i][k] for k in range(budget_bits)]) for i in range(n)) <= bb)

# import the classical optimizer
from qiskit.aqua.components.optimizers import COBYLA

optim_dict = {
  "docplex_mod": mdl,
  "quantum_instance": 'qasm_simulator',
  "shots": 1024,
  "print": True,
  "solver":'vqe',
  "optimizer":COBYLA,
  "maxiter":1000,
  "depth":1,
  "alpha":0.35
}

In [9]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

2021-03-27 09:55:09,685:qiskit.aqua.quantum_instance:INFO: 
Qiskit Terra version: 0.16.4
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['ccx', 'cp', 'cswap', 'csx', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'delay', 'diagonal', 'h', 'id', 'initialize', 'kraus', 'mcp', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcy', 'mcz', 'multiplexer', 'p', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'snapshot', 'swap', 'sx', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z'], 'coupling_map': None}
{'initial_layout': None, 'seed_transpiler': None, 'optimization_level': None}
RunConfig(max_credits=10, shots=1024)
{'timeout': None}
{}
{}
Measurement mitigation: None
### Original problem:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio_optimization

Maximize
 obj: 0.002900000000 y0_0 + 0.005800000000 y0_1 + 0.011600000000 y0_2
      + 0.023200000000 y0_3 

In [10]:
results['result'].variables_dict

{'y0_0': 0.0,
 'y0_1': 1.0,
 'y0_2': 0.0,
 'y0_3': 0.0,
 'y1_0': 1.0,
 'y1_1': 1.0,
 'y1_2': 0.0,
 'y1_3': 1.0,
 'y2_0': 0.0,
 'y2_1': 0.0,
 'y2_2': 0.0,
 'y2_3': 0.0,
 'y3_0': 0.0,
 'y3_1': 1.0,
 'y3_2': 0.0,
 'y3_3': 0.0,
 'y4_0': 0.0,
 'y4_1': 0.0,
 'y4_2': 0.0,
 'y4_3': 0.0}

In [11]:
print('Binary variables')
ms = np.array(list(results['result'].variables_dict.values())).astype(int)
print(ms)

print()
print('Binary results(reverse order): y')
y_val= np.zeros([n, budget_bits]).astype(int)
y_val=[[ms[i+j*budget_bits] for i in range(budget_bits)] for j in range(n)]
print(y_val)

print()
print('Integer results (amount of groups of stocks): x')
x_val= np.zeros(n).astype(int)
for i, a in enumerate(y_val):
    x_val[i] = (np.sum([(2**k)*a[k] for k in range(budget_bits)]))
print(x_val)

print()
print('Amount of individual stock:')
x_ind = grouping*x_val
print(x_ind)

print()
print('Amount invest in each stock:')
invest = s_real*x_ind
print(invest)

print()
print('Total invest:')
print(invest.sum())

Binary variables
[0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0]

Binary results(reverse order): y
[[0, 1, 0, 0], [1, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0]]

Integer results (amount of groups of stocks): x
[ 2 11  0  2  0]

Amount of individual stock:
[ 4 33  0  8  0]

Amount invest in each stock:
[ 240.16 1843.71    0.    259.68    0.  ]

Total invest:
2343.5499999999997


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table

In [13]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.16.4',
 'qiskit-aer': '0.7.6',
 'qiskit-ignis': '0.5.2',
 'qiskit-ibmq-provider': '0.12.1',
 'qiskit-aqua': '0.8.2',
 'qiskit': '0.24.0'}